In [1]:
#| default_exp ocr_idefics

In [2]:
#| hide
# %reload_ext autoreload
# %autoreload 0


# install (Colab)

In [3]:
# try: 
#     import fastcore as FC
# except ImportError: 
#     !pip install -q fastcore
# try:
#     import rich
# except ImportError:
#     !pip install -q rich


In [4]:
# !pip install -q git+https://github.com/civvic/PanelCleaner.git@basic-tesseract

need version >4.40 of transformers

In [5]:
# %pip install git+https://github.com/huggingface/transformers

Fash attention doesn't support Metal [#412](https://github.com/Dao-AILab/flash-attention/issues/412) (but see [metal-flash-attention](https://github.com/philipturner/metal-flash-attention))



In [6]:
# %env FLASH_ATTENTION_SKIP_CUDA_BUILD=TRUE
# %pip install flash-attn --no-build-isolation

In [7]:
!nvidia-smi

Fri May 10 18:32:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090 Ti     On  | 00000000:65:00.0 Off |                  Off |
|  0%   50C    P8              33W / 480W |      1MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Testing `Idefics` OCR for Comics
> Accuracy Enhancements for OCR in `PanelCleaner`


# Prologue

In [8]:
#| export
from __future__ import annotations

import functools
from pathlib import Path

import pcleaner.ocr.ocr as ocr
import torch
import transformers
from pcleaner.ocr.ocr_tesseract import TesseractOcr
from PIL import Image
from rich.console import Console
from transformers import AutoProcessor
from transformers import Idefics2ForConditionalGeneration
from transformers import PreTrainedModel


In [9]:
import sys
from typing import cast

import fastcore.xtras  # patch Path with some utils
import pcleaner.config as cfg
from fastcore.test import *  # type: ignore


In [10]:
transformers.__version__

'4.41.0.dev0'

# Helpers

In [11]:
# pretty print by default
# %load_ext rich

In [12]:
#| exporti

console = Console(width=104, tab_size=4, force_jupyter=True)
cprint = console.print


Force reload of `experiments` module

In [13]:
if 'experiments' in sys.modules:
    import importlib; importlib.reload(experiments)  # type: ignore
else:
    import experiments


In [14]:
#| export

from experiments import *
from helpers import *
from ocr_metric import *


In [15]:
#| exporti

def load_image(img_or_path) -> Image.Image:
    if isinstance(img_or_path, (str, Path)):
        return Image.open(img_or_path)
    elif isinstance(img_or_path, Image.Image):
        return img_or_path
    else:
        raise ValueError(f"img_or_path must be a path or PIL.Image, got: {type(img_or_path)}")


----
# Idefics basic usage

not working, cuda memory error

In [16]:
# # Note that passing the image urls (instead of the actual pil images) to the processor is also possible
# # image1 = load_image("https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg")
# # image2 = load_image("https://cdn.britannica.com/59/94459-050-DBA42467/Skyline-Chicago.jpg")
# # image3 = load_image("https://cdn.britannica.com/68/170868-050-8DDE8263/Golden-Gate-Bridge-San-Francisco.jpg")

# image1 = Image.open("media/Statue-of-Liberty-Island-New-York-Bay.webp")
# image2 = Image.open("media/Skyline-Chicago.webp")
# image3 = Image.open("media/Golden-Gate-Bridge-San-Francisco.webp")

In [17]:
# processor = AutoProcessor.from_pretrained("HuggingFaceM4/idefics2-8b")

In [18]:
# model = Idefics2ForConditionalGeneration.from_pretrained(
#         "HuggingFaceM4/idefics2-8b",
#         torch_dtype=torch.bfloat16,
#         #_attn_implementation="flash_attention_2",
#         )


In [19]:
# assert isinstance(model, PreTrainedModel)
# model.to(DEVICE)
# type(model), model.device


Create inputs

In [20]:
# messages = [
#     {
#         "role": "user",
#         "content": [
#             {"type": "image"},
#             {"type": "text", "text": "What do we see in this image?"},
#         ]
#     },
#     {
#         "role": "assistant",
#         "content": [
#             {"type": "text", "text": "In this image, we can see the city of New York, and more specifically the Statue of Liberty."},
#         ]
#     },
#     {
#         "role": "user",
#         "content": [
#             {"type": "image"},
#             {"type": "text", "text": "And how about this image?"},
#         ]
#     },       
# ]


In [21]:
# prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
# inputs = processor(text=prompt, images=[image1, image2], return_tensors="pt")
# inputs = {k: v.to(DEVICE) for k, v in inputs.items()}


Generate

In [22]:
# generated_ids = model.generate(**inputs, max_new_tokens=500)
# generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)

# print(generated_texts)
# # ['User: What do we see in this image? \nAssistant: In this image, we can see the city of New York, and more specifically the Statue of Liberty. \nUser: And how about this image? \nAssistant: In this image we can see buildings, trees, lights, water and sky.']


In [23]:
# [
#     'User: What do we see in this image? '
#     'Assistant: In this image, we can see the city of New York, and more specifically the Statue of Liberty. '
#     'User: And how about this image? '
#     'Assistant: In this image we can see buildings, trees, lights, water and sky.'
#     ]

----
# Idefics experiments


# Idefics

## Idefics initialization

In [24]:
#| exporti

processor = AutoProcessor.from_pretrained("HuggingFaceM4/idefics2-8b")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
#| exporti

device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"

model = Idefics2ForConditionalGeneration.from_pretrained(
        "HuggingFaceM4/idefics2-8b",
        torch_dtype=torch.bfloat16,
        _attn_implementation="flash_attention_2",
        ).to(device)  # type: ignore


You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [26]:
type(model), model.device


(transformers.models.idefics2.modeling_idefics2.Idefics2ForConditionalGeneration,
 device(type='cuda', index=0))

In [27]:
#| exporti

prompt_text_tmpl = (
        "Please perform optical character recognition (OCR) on this image, which displays "
        "speech balloons from a comic book. The text is in {}. Extract the text and "
        "format it as follows: transcribe in standard sentence case, avoid using all capital "
        "letters. Provide the transcribed text clearly and double check the sentence is not all capital letters.")

# prompt_text_tmpl = ("Please perform optical character recognition (OCR) on this image, which displays "
#         f"speech balloons from a manga comic. The text is in {}. Extract the text and "
#         "format it without newlines. Provide the transcribed text clearly.")

# prompt_text_tmpl = ("Please perform optical character recognition (OCR) on this image, which displays "
#         "speech balloons from a comic book. The text is in {}. Extract the text and "
#         "format it as follows: transcribe in standard sentence case (avoid using all capital "
#         "letters) and use asterisks to denote any words that appear in bold within the image. "
#         "Provide the transcribed text clearly.")

# prompt_text_tmpl = ("Please perform optical character recognition (OCR) on this image, which displays "
#         "speech balloons from a comic book. The text is in {}. Extract the text and "
#         "format it as follows: transcribe in standard sentence case, capitalized. Avoid using "
#         "all capital letters. In comics, it is common to use two hyphens '--' to interrupt a sentence. "
#         "Retain any hyphens as they appear in the original text. Provide the transcribed text "
#         "clearly, ensuring it is capitalized where appropriate, including proper nouns.")

prompt_text_tmpl = (
        "Please perform optical character recognition (OCR) on this image, which displays "
        "speech balloons from a comic book. The text is in {}. Extract the text and "
        "format it as follows: transcribe in standard sentence case, capitalized. Avoid using "
        "all capital letters, but ensure it is capitalized where appropriate, including proper nouns. "
        "Provide the transcribed text clearly. Double check the text is not all capital letters.")

default_prompt_text_tmpl = prompt_text_tmpl

## IdeficsOCR

In [28]:
#| export

class IdeficsOCR:
    prompt_text_tmpl: str = default_prompt_text_tmpl

    def __init__(self, 
            lang: str | None = None, 
            prompt_text_tmpl: str|None = None, 
            device: str | None = None
        ):
        self.lang = lang
        self.prompt_text_tmpl = prompt_text_tmpl or self.prompt_text_tmpl
        self.device = (device or 
            "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

    @staticmethod
    def is_idefics_available() -> bool:
        return True

    def _generation_args(self, image: Image.Image, resulting_messages: list[dict]):
        prompt = processor.apply_chat_template(resulting_messages, add_generation_prompt=True)
        inputs = processor(text=prompt, images=[image], return_tensors="pt")
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        max_new_tokens = 512
        repetition_penalty = 1.2
        decoding_strategy = "Greedy"
        temperature = 0.4
        top_p = 0.8

        generation_args = {
            "max_new_tokens": max_new_tokens,
            "repetition_penalty": repetition_penalty,
        }

        assert decoding_strategy in [
            "Greedy",
            "Top P Sampling",
        ]

        if decoding_strategy == "Greedy":
            generation_args["do_sample"] = False
        elif decoding_strategy == "Top P Sampling":
            generation_args["temperature"] = temperature
            generation_args["do_sample"] = True
            generation_args["top_p"] = top_p

        generation_args.update(inputs)
        return prompt, generation_args

    def __call__(
        self,
        img_or_path: Image.Image | Path | str,
        prompt_text: str | None = None,
        lang: str | None = None,
        config: str | None = None,
        show_prompt: bool = False,
        **kwargs,
    ) -> str:
        if not self.is_idefics_available():
            raise RuntimeError("Idefics is not installed or not found.")
        resulting_messages = [
            {
                "role": "user",
                "content": [{"type": "image"}] + [
                    {"type": "text", "text": prompt_text or self.prompt_text_tmpl.format(lang or self.lang)}
                ]
            }
        ]
        image = load_image(img_or_path)
        prompt, generation_args = self._generation_args(image, resulting_messages)
        generated_ids = model.generate(**generation_args)
        generated_texts = processor.batch_decode(
            generated_ids[:, generation_args["input_ids"].size(1):], skip_special_tokens=True)
        if show_prompt:
            cprint("INPUT:", prompt, "|OUTPUT:", generated_texts)
        return generated_texts[0]#.strip('"')

    def postprocess_ocr(self, text):
        return ' '.join(remove_multiple_whitespaces(text).splitlines())


## IdeficsExperimentContext

In [29]:
#| export

class IdeficsExperimentContext(OCRExperimentContext):
    @functools.lru_cache()
    def mocr(self, ocr_model: str, lang: str):
        if ocr_model == 'Idefics':
            proc = IdeficsOCR(lang)
        else:
            engine = self.engines[ocr_model]
            ocr_processor = ocr.get_ocr_processor(True, engine)
            proc = ocr_processor[lang2pcleaner(lang)]
            if isinstance(proc, TesseractOcr):
                proc.lang = lang2tesseract(lang)
        return proc


# PanelCleaner Configuration
> Adapt `PanelCleaner` `Config` current config to this notebook.


In [30]:
config = cfg.load_config()
config.cache_dir = Path(".")

cache_dir = config.get_cleaner_cache_dir()

profile = config.current_profile
preprocessor_conf = profile.preprocessor
# Modify the profile to OCR all boxes.
# Make sure OCR is enabled.
preprocessor_conf.ocr_enabled = True
# Make sure the max size is infinite, so no boxes are skipped in the OCR process.
preprocessor_conf.ocr_max_size = 10**10
# Make sure the sus box min size is infinite, so all boxes with "unknown" language are skipped.
preprocessor_conf.suspicious_box_min_size = 10**10
# Set the OCR blacklist pattern to match everything, so all text gets reported in the analytics.
preprocessor_conf.ocr_blacklist_pattern = ".*"


# Test images
> `IMAGE_PATHS` is a list of image file paths that are used as input for testing the OCR methods.

In [31]:
media_path = Path("media/")

IMAGE_PATHS = sorted(
    [_ for _ in media_path.glob("*") if _.is_file() and _.suffix.lower() in [".jpg", ".png", ".jpeg"]])

[f"{i:02}: {_.name}" for i,_ in enumerate(IMAGE_PATHS)]


['00: Action_Comics_1960-01-00_(262).JPG',
 '01: Adolf_Cap_01_008.jpg',
 '02: Barnaby_v1-028.png',
 '03: Barnaby_v1-029.png',
 '04: Buck_Danny_-_12_-_Avions_Sans_Pilotes_-_013.jpg',
 '05: Cannon-292.jpg',
 '06: Contrato_con_Dios_028.jpg',
 '07: Erase_una_vez_en_Francia_02_88.jpg',
 '08: FOX_CHILLINTALES_T17_012.jpg',
 '09: Furari_-_Jiro_Taniguchi_selma_056.jpg',
 '10: Galactus_12.jpg',
 '11: INOUE_KYOUMEN_002.png',
 '12: MCCALL_ROBINHOOD_T31_010.jpg',
 '13: MCCAY_LITTLENEMO_090.jpg',
 '14: Mary_Perkins_On_Stage_v2006_1_-_P00068.jpg',
 '15: PIKE_BOYLOVEGIRLS_T41_012.jpg',
 '16: Sal_Buscema_Spaceknights_&_Superheroes_Ocular_Edition_1_1.png',
 '17: Sal_Buscema_Spaceknights_&_Superheroes_Ocular_Edition_1_1_K.png',
 '18: Sal_Buscema_Spaceknights_&_Superheroes_Ocular_Edition_1_2.png',
 '19: Spirou_Et_Fantasio_Integrale_06_1958_1959_0025_0024.jpg',
 '20: Strange_Tales_172005.jpg',
 '21: Strange_Tales_172021.jpg',
 '22: Tarzan_014-21.JPG',
 '23: Tintin_21_Les_Bijoux_de_la_Castafiore_page_39.jp

# CONTEXT


In [32]:
CONTEXT = IdeficsExperimentContext(None, IMAGE_PATHS)

gpu = torch.cuda.is_available() or torch.backends.mps.is_available()
model_path = CONTEXT.config.get_model_path(gpu)
DEVICE = ("mps" if torch.backends.mps.is_available() else "cuda") if model_path.suffix == ".pt" else "cpu"

CONTEXT.config.show()
cprint(
    f"{'cache_dir':>15}: {repr(cache_dir)}\n"
    f"{'model_path':>15}: {repr(model_path)}\n"
    f"{'device':>15}: {repr(DEVICE)}")


Current Configuration:

Locale: System default
Default Profile: Built-in
Saved Profiles:
- victess: /home/vic/dev/repo/DL-mac/cleaned/victess.conf
- victmang: /home/vic/dev/repo/DL-mac/cleaned/vicmang.conf

Profile Editor: System default
Cache Directory: .
Default Torch Model Path: /home/vic/.cache/pcleaner/model/comictextdetector.pt
Default CV2 Model Path: /home/vic/.cache/pcleaner/model/comictextdetector.pt.onnx
GUI Theme: System default

--------------------

Config file located at: /home/vic/.config/pcleaner/pcleanerrc
System default cache directory: /home/vic/.cache/pcleaner


cache_dir: Path('cleaner')
     model_path: Path('/home/vic/.cache/pcleaner/model/comictextdetector.pt')
         device: 'cuda'

# Base image
> Change `BASE_IMAGE_IDX` to select a different base image to use in the examples below.

In [33]:
BASE_IMAGE_IDX: ImgIdT = cast(ImgIdT, CONTEXT.normalize_idx("Strange_Tales_172005.jpg"))
assert CONTEXT.path_from_idx(BASE_IMAGE_IDX).exists()


# Visualize images


In [34]:
IMAGE_CONTEXT = ImageContext(CONTEXT, BASE_IMAGE_IDX)


In [35]:
img_visor = ImageContextVisor(CONTEXT, BASE_IMAGE_IDX)
img_visor


Output(layout=Layout(height='0px'))

Output()

# Box id
> change `BOX_IDX` to use any box to test crop methods

In [36]:
BOX_IDX = 0

# Idefics inference

In [37]:
page_lang = IMAGE_CONTEXT.page_lang

resulting_messages = [
    {
        "role": "user",
        "content": [{"type": "image"}] + [
            {"type": "text", "text": prompt_text_tmpl.format(page_lang)}
        ]
    }
]

In [38]:
def idefics_generation_args(image: Image.Image, resulting_messages: list[dict]):
    prompt = processor.apply_chat_template(resulting_messages, add_generation_prompt=True)
    inputs = processor(text=prompt, images=[image], return_tensors="pt")
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
    
    max_new_tokens = 512
    repetition_penalty = 1.2
    decoding_strategy = "Greedy"
    temperature = 0.4
    top_p = 0.8

    generation_args = {
        "max_new_tokens": max_new_tokens,
        "repetition_penalty": repetition_penalty,
    }

    assert decoding_strategy in [
        "Greedy",
        "Top P Sampling",
    ]

    if decoding_strategy == "Greedy":
        generation_args["do_sample"] = False
    elif decoding_strategy == "Top P Sampling":
        generation_args["temperature"] = temperature
        generation_args["do_sample"] = True
        generation_args["top_p"] = top_p

    generation_args.update(inputs)
    return prompt, generation_args


### Crop methods

In [39]:
# image_experiment = ExperimentOCR(IMAGE_CONTEXT, 'Idefics')
image_experiment = ExperimentOCR.from_image(CONTEXT, 'Idefics', IMAGE_CONTEXT.image_idx)  # use cache


In [40]:
method = CropMethod.INITIAL_BOX

result = cast(ResultOCR, image_experiment.result(BOX_IDX, method, ocr=False))
image = cast(Image.Image, result.image)


In [41]:
prompt, generation_args = idefics_generation_args(image, resulting_messages)
generated_ids = model.generate(**generation_args)

generated_texts = processor.batch_decode(
    generated_ids[:, generation_args["input_ids"].size(1):], skip_special_tokens=True)
cprint("INPUT:", prompt, "|OUTPUT:", generated_texts)


No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


INPUT: User:<image>Please perform optical character recognition (OCR) on this image, which displays 
speech balloons from a comic book. The text is in English. Extract the text and format it as follows: 
transcribe in standard sentence case, capitalized. Avoid using all capital letters, but ensure it is 
capitalized where appropriate, including proper nouns. Provide the transcribed text clearly. Double 
check the text is not all capital letters.<end_of_utterance>
Assistant: |OUTPUT:
[
    'Embowered by great gnarled cypress trees, the ancient manor stands alone on the outskirts of New 
Orleans, kept tidy by a white-haired old man known only as Bambu.'
]

In [42]:
result.ocr = generated_texts[0]
result


,"Embowered by great gnarled cypress trees, the ancient manor stands alone on the outskirts of New Orleans, kept tidy by a white-haired old man known only as Bambu.1.00"


----

In [43]:
method = CropMethod.INITIAL_BOX

result = cast(ResultOCR, image_experiment.result(BOX_IDX, method, ocr=False))
image = cast(Image.Image, result.image)

mocr: IdeficsOCR = cast(IdeficsOCR, CONTEXT.mocr('Idefics', page_lang))
text = mocr(image, show_prompt=True)
result.ocr = mocr.postprocess_ocr(text)
result


INPUT: User:<image>Please perform optical character recognition (OCR) on this image, which displays 
speech balloons from a comic book. The text is in English. Extract the text and format it as follows: 
transcribe in standard sentence case, capitalized. Avoid using all capital letters, but ensure it is 
capitalized where appropriate, including proper nouns. Provide the transcribed text clearly. Double 
check the text is not all capital letters.<end_of_utterance>
Assistant: |OUTPUT:
[
    'Embowered by great gnarled cypress trees, the ancient manor stands alone on the outskirts of New 
Orleans, kept tidy by a white-haired old man known only as Bambu.'
]

,"Embowered by great gnarled cypress trees, the ancient manor stands alone on the outskirts of New Orleans, kept tidy by a white-haired old man known only as Bambu.1.00"


In [44]:
image_experiment.result(BOX_IDX, CropMethod.PADDED_4)

,"Embowered by great gnarled cypress trees, the ancient manor stands alone on the outskirts of new orleans, kept tidy by a white-haired old man known only as bambu.0.98"


In [45]:
image_experiment.result(BOX_IDX, CropMethod.PAD_8_FRACT_0_2)

,,


----
# Visualize results

In [46]:
result_visor = ResultVisor(image_experiment)
result_visor


Output(layout=Layout(height='0px'))

Output()

----
# Visualize Experiment

In [47]:
# p, d = image_experiment.to_json()

In [48]:
exp_visor = ExperimentVisor(image_experiment)
exp_visor


Output()

In [49]:
# p, d = exp_visor.ctx.to_json()
# p

----
# EEAaO

In [54]:
idefics_experiment = ExperimentsVisor(CONTEXT, BASE_IMAGE_IDX, 
                            box_idx=13, method=CropMethod.DEFAULT_GREY_PAD,
                            ocr_model=OCRModel.IDEFICS, 
                            ocr_models={'Tesseract': OCRModel.TESSERACT, 'Idefics': OCRModel.IDEFICS})
idefics_experiment


Output()

In [53]:
idefics_experiment.update(model=OCRModel.TESSERACT)

# Colophon
----


In [56]:
import fastcore.all as FC
from nbdev.export import nb_export


In [57]:
if FC.IN_NOTEBOOK:
    nb_export('test_idefics.ipynb', '.')
